<a href="https://colab.research.google.com/github/phmouras/Critical-Phenomena-in-gravitational-collapse-via-spectral-methods/blob/main/Origin_MSSC_N500_filter_Potential_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
%matplotlib inline
mpl.style.use('ggplot')


N = 300                                                   # Truncation ordem

L0 = 15                                             # Map parameter

col = np.cos(np.arange(2*N + 3)*math.pi /(2*N + 2))      # collocation points (Verificado)

colr = col[1:N+2]                                                         # physical domain (Verificado)

r1 = L0 * colr/(np.sqrt(1-colr**2))
r = np.flip(r1)                                           # physical domain (Verificado)

#out_r = open('r_30_L02.txt', 'a')
#out_r.write(' ' +' '.join(str('%.18f'%n) for n in r)+'\n')
#out_r.close()


# Base Matrix (Tchebyshev Polinomials):

SB = np.zeros([N+2,N+1])
rSB = np.zeros([N+2,N+1])
rrSB = np.zeros([N+2,N+1])


for i in range(N+1+1):                                    # SB[:,0] = function in origin
  SB[i,] = np.sin((2*i+1)*np.arctan(L0/r))

for i in range(N+1+1):
  rSB[i,] = -np.cos((2*i+1)*np.arctan(L0/r))*(2*i+1)*L0/(r**2*(1+L0**2/r**2))

for i in range(N+1+1):
  rrSB[i,] = -np.sin((2*i+1)*np.arctan(L0/r))*(2*i+1)**2*L0**2/(r**4*(1+L0**2/r**2)**2)+2*np.cos((2*i+1)*np.arctan(L0/r))*(2*i+1)*L0/(r**3*(1+L0**2/r**2))-2*np.cos((2*i+1)*np.arctan(L0/r))*(2*i+1)*L0**3/(r**5*(1+L0**2/r**2)**2)

# Base function



psi = SB[0:N+1,:]        # origin included in psi[:,0]
rpsi = rSB[0:N+1,:]
rrpsi = rrSB[0:N+1,:]



# Initial conditions of Phi (Scalar field)

#r0 = 0

#sigma = 1/2

# L0 = 1 and sigma = 1/2 and Phi_init = A0*(np.exp(-(r-r0)**2/sigma**2) + np.exp(-(r+r0)**2/sigma**2))

#A0 =  0.76  #  0.7602785  -> disperse ;  # 0.77 -> colapse

# L0 = 3 and sigma = 1/2 and Phi_init = A0*(np.exp(-(r-r0)**2/sigma**2) + np.exp(-(r+r0)**2/sigma**2))

#A0 =  0.75981136951800  #  0.759811369518  -> disperse ;  # 0.759811369518001   -> colapse

Phi_c = 1/2

Phi_vac = -1/2

R0 = 3.5  # 0.05 -> disperse ;  # 0.  -> colapse   A0 substitution

Phi_init = (Phi_c - Phi_vac)*np.exp(-r**2/R0**2) + Phi_vac          # Phi initial data out of origin (Verificado)

inv_psi = np.linalg.inv(psi)

a0 = np.dot(Phi_init, inv_psi)  # coeficients a(0)  (Verificado)

#out_a = open('a0_30_L02.txt', 'a')
#out_a.write(' ' +' '.join(str('%.18f'%n) for n in a0)+'\n')
#out_a.close()


Phi = np.dot(a0, psi)        # approximative solution in t = 0
rPhi= np.dot(a0, rpsi)

########################### Plot: Initial Conditions of Phi

M = 3000       # plot truncation

rplot = np.linspace(0.00000000000000001,15,M)

colplot = rplot/np.sqrt(L0**2 + rplot**2)

SBplot = np.zeros([N+1,M])
rSBplot = np.zeros([N+1,M])
rrSBplot = np.zeros([N+1,M])

for i in range(N+1):
  SBplot[i,] = np.sin((2*i+1)*np.arctan(L0/rplot))

for i in range(N+1):
  rSBplot[i,] = -np.cos((2*i+1)*np.arctan(L0/rplot))*(2*i+1)*L0/(rplot**2*(1+L0**2/rplot**2))

for i in range(N+1):
  rrSBplot[i,] = -np.sin((2*i+1)*np.arctan(L0/rplot))*(2*i+1)**2*L0**2/(rplot**4*(1+L0**2/rplot**2)**2)+2*np.cos((2*i+1)*np.arctan(L0/rplot))*(2*i+1)*L0/(rplot**3*(1+L0**2/rplot**2))-2*np.cos((2*i+1)*np.arctan(L0/rplot))*(2*i+1)*L0**3/(rplot**5*(1+L0**2/rplot**2)**2)

psiplot = SBplot[0:(N+1),:]
rpsiplot = rSBplot[0:(N+1),:]
rrpsiplot = rrSBplot[0:(N+1),:]

Phiplot_init = (Phi_c - Phi_vac)*np.exp(-rplot**2/R0**2) + Phi_vac

#Phiplot_init = np.hstack((0, Phiplot_int))

Phiplot = np.dot(a0, psiplot)

plt.plot(rplot, Phiplot, rplot, Phiplot_init, "--")   #(Verificado)
plt.xlabel('r')
plt.xlim(0,8)
plt.show()


In [2]:
erro = Phiplot_init - Phiplot

plt.plot(rplot, erro)
plt.xlabel('r')
plt.xlim(0,8)
plt.ylabel("$(\phi_N - \phi_0)$")
plt.show()

In [3]:

# Plot 3D from Phi initial:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122,projection='3d')

y = Phiplot
x = rplot
theta = np.linspace(0, 2*np.pi, M)    # Revolution of f(phi,r)

xn = np.outer(x, np.cos(theta))
yn = np.outer(x, np.sin(theta))
zn = np.zeros_like(xn)

for i in range(len(x)):
    zn[i,:] = np.full_like(zn[0], y[i])

ax1.plot(x, y)
ax2.plot_surface(xn, yn, zn)
plt.show()

In [4]:
Pi_0 = np.zeros(N+1)
b0 = np.dot(Pi_0, psi)
Pi = np.dot(b0, psi)

c0 = np.zeros([N+1])     # guess value
for i in range(N+1):
  c0[i] = 0.0001

def V(Phi):
  return 1/4*(Phi**2 - Phi_vac**2)**2

def dV(Phi):
  return Phi**3 - Phi_vac**2*Phi


coeforigin = 1/L0**2*(-(-1)**(np.arange(N+1))*(2*np.arange(1,N+2)-1)**2)    # 1/r * d\phi/dr

tol = 1e-18
#cnew = 0
#err = abs(c0 - cnew)

n = 0
nf = 500

# Newton Raphson loop
#while max(err) >= tol:
while n <= nf:
  Chi = np.dot(c0, psi)
  rChi = np.dot(c0 ,rpsi)
  rrChi = np.dot(c0, rrpsi)
  H0_0 = 4*rrChi[0] + 8*np.dot(coeforigin,c0) + 1/2*rPhi[0]**2 + np.exp(4*Chi[0])*V(Phi)[0]
  H0_int = 4*rChi[1:N+1]**2 + 4*rrChi[1:N+1] + 8/r[1:N+1]*rChi[1:N+1] + 1/2*rPhi[1:N+1]**2 + np.exp(4*Chi[1:N+1])*V(Phi)[1:N+1]
  H0 = np.hstack((H0_0, H0_int))
  J1 = rChi*rpsi
  JH_0 = 8*J1[:,0]+ 4*rrpsi[:,0] + 8*coeforigin + 4*np.exp(4*Chi[0])*rpsi[:,0]*V(Phi)[0]
  JH_int = 8*J1[:,1:N+1] + 4*rrpsi[:,1:N+1] + 8/r[1:N+1]*rpsi[:,1:N+1] + 4*np.exp(4*Chi[1:N+1])*rpsi[:,1:N+1]*V(Phi)[1:N+1]
  JH = np.column_stack((JH_0, JH_int))
  inv_JH = np.linalg.inv(JH)
  cnew = c0
  c0 = c0 - np.dot(H0, inv_JH)
  err = abs(c0 - cnew)
  if  max(err) <= tol:
    print('n=', n, 'Erro_max=', max(err), 'c0=', c0, sep = '\n')

    break
  else:
    n = n + 1


n=
205
Erro_max=
4.336808689942018e-19
c0=
[ 7.93052326e-02 -2.52331495e-02  1.44889128e-02 -4.36931655e-03
  2.45000535e-03  9.46885664e-04 -3.50032722e-04  1.53346111e-03
 -3.85774400e-04  9.77887174e-04  2.12479054e-05  5.01813318e-04
  2.40620724e-04  2.87608776e-04  2.78740813e-04  2.20822733e-04
  2.48989649e-04  2.00211203e-04  2.12949433e-04  1.86325858e-04
  1.85789604e-04  1.71852805e-04  1.66279924e-04  1.57675149e-04
  1.51314482e-04  1.44852948e-04  1.39035222e-04  1.33632992e-04
  1.28558264e-04  1.23865489e-04  1.19443422e-04  1.15314382e-04
  1.11425624e-04  1.07766996e-04  1.04315608e-04  1.01053261e-04
  9.79666725e-05  9.50396973e-05  9.22617899e-05  8.96204661e-05
  8.71065278e-05  8.47103662e-05  8.24240312e-05  8.02398897e-05
  7.81511987e-05  7.61517127e-05  7.42357573e-05  7.23981241e-05
  7.06340323e-05  6.89390886e-05  6.73092438e-05  6.57407626e-05
  6.42301932e-05  6.27743395e-05  6.13702400e-05  6.00151443e-05
  5.87064966e-05  5.74419169e-05  5.62191878e-0

In [5]:

Chiplot = np.dot(c0, psiplot)


plt.plot(rplot,Chiplot)
plt.xlim(0,8)
plt.show()


In [6]:
# Equation for Krr: momentum constraint

# Initial values of Krr: Base functions of Krr

# In r = 0:

SB1 = 1/2*(SB[1:(N+1),:] + SB[0:(N),:])          # VERIFICADO
rSB1 = 1/2*(rSB[1:(N+1),:] + rSB[0:(N),:])
rrSB1 = 1/2*(rrSB[1:(N+1),:] + rrSB[0:(N),:])

# Base functions for Beta

SB2 = np.zeros([N,N+1])
rSB2 = np.zeros([N,N+1])
rrSB2 = np.zeros([N,N+1])



for i in range(N):                                                   # VERIFICADO
  SB2[i,] = np.sin((2*(i+1/2)+1)*np.arctan(L0/r))


for i in range(N):
  rSB2[i,] = -np.cos((2*i+2)*np.arctan(L0/r))*(2*i+2)*L0/(r**2*(1+L0**2/r**2))


for i in range(N):
  rrSB2[i,] = -np.sin((2*i+2)*np.arctan(L0/r))*(2*i+2)**2*L0**2/(r**4*(1+L0**2/r**2)**2)+2*np.cos((2*i+2)*np.arctan(L0/r))*(2*i+2)*L0/(r**3*(1+L0**2/r**2))-2*np.cos((2*i+2)*np.arctan(L0/r))*(2*i+2)*L0**3/(r**5*(1+L0**2/r**2)**2)






In [7]:
np.shape(SB2)

(300, 301)

In [8]:
np.shape(SB1)

(300, 301)

In [9]:
c0

array([ 7.93052326e-02, -2.52331495e-02,  1.44889128e-02, -4.36931655e-03,
        2.45000535e-03,  9.46885664e-04, -3.50032722e-04,  1.53346111e-03,
       -3.85774400e-04,  9.77887174e-04,  2.12479054e-05,  5.01813318e-04,
        2.40620724e-04,  2.87608776e-04,  2.78740813e-04,  2.20822733e-04,
        2.48989649e-04,  2.00211203e-04,  2.12949433e-04,  1.86325858e-04,
        1.85789604e-04,  1.71852805e-04,  1.66279924e-04,  1.57675149e-04,
        1.51314482e-04,  1.44852948e-04,  1.39035222e-04,  1.33632992e-04,
        1.28558264e-04,  1.23865489e-04,  1.19443422e-04,  1.15314382e-04,
        1.11425624e-04,  1.07766996e-04,  1.04315608e-04,  1.01053261e-04,
        9.79666725e-05,  9.50396973e-05,  9.22617899e-05,  8.96204661e-05,
        8.71065278e-05,  8.47103662e-05,  8.24240312e-05,  8.02398897e-05,
        7.81511987e-05,  7.61517127e-05,  7.42357573e-05,  7.23981241e-05,
        7.06340323e-05,  6.89390886e-05,  6.73092438e-05,  6.57407626e-05,
        6.42301932e-05,  

In [10]:
Nq = int(3/2*N)           # Quadrature truncation

gauss_quadrature = np.polynomial.legendre.leggauss(Nq + 1)

new_col = gauss_quadrature[0]            # Legendre quadrature points


# Legendre Polinomials

P = np.zeros([Nq+3,Nq+1])
colP = np.zeros([Nq+3,Nq+1])

P[0,] = 1
P[1,] = new_col

colP[0,] = 0
colP[1,] = 1

for i in range(2,Nq+3):
  P[i,] = ((2*i-1)*new_col*P[i-1,] - (i-1)*P[i-2,])/(i)

for i in range(2,Nq+3):
  colP[i,] = i*P[i-1] + new_col*colP[i-1]

P_max = P[Nq+1]

colP_max = colP[Nq+1]

wq_col = 2/((1-new_col**2)*colP_max**2)    # Legendre weight (Verificado)

rq = L0*(1+new_col)/(1-new_col)            # Physical quadrature domain

qSB = np.zeros([Nq+1,Nq+1])
qrSB = np.zeros([Nq+1,Nq+1])
qrrSB = np.zeros([Nq+1,Nq+1])


for i in range(Nq+1):
  qSB[i,] = np.sin((2*i+1)*np.arctan(L0/rq))

for i in range(Nq+1):
  qrSB[i,] = -np.cos((2*i+1)*np.arctan(L0/rq))*(2*i+1)*L0/(rq**2*+L0**2)

for i in range(Nq+1):
  qrrSB[i,] = -np.sin((2*i+1)*np.arctan(L0/rq))*(2*i+1)**2*L0**2/(rq**4*(1+L0**2/rq**2)**2)+2*np.cos((2*i+1)*np.arctan(L0/rq))*(2*i+1)*L0/(rq**3*(1+L0**2/rq**2))-2*np.cos((2*i+1)*np.arctan(L0/rq))*(2*i+1)*L0**3/(rq**5*(1+L0**2/rq**2)**2)

qpsi = qSB[0:N+1,:]
rqpsi = qrSB[0:N+1,:]
rrqpsi = qrrSB[0:N+1,:]


qSB1 = 1/2*(qSB[1:(N+1),:] + qSB[0:(N),:])          # VERIFICADO
rqSB1 = 1/2*(qrSB[1:(N+1),:] + qrSB[0:(N),:])
rrqSB1 = 1/2*(qSB[1:(N+1),:] + qrrSB[0:(N),:])



# Initial Phi in quadrature points

#qPhi = np.dot(a0, qpsi)
#rqPhi= np.dot(a0, rqpsi)

# Initial Pi for quadrature points

#qPi = np.dot(b0, qpsi)


# Initial Chi for quadrature points:

#qChi = np.dot(c0, qpsi)   # Verificado todos
#rqChi = np.dot(c0, rqpsi)
#rrqChi = np.dot(c0, rrqpsi)

# Initial values of Krr:

#qKrr = np.dot(ck0, qSB1)

# Alpha na origem

#Alphacenter = (-1)**(np.arange(N+1))


In [11]:
# Filtering

Nc = 0

Nf = N - Nc

coef_f = 36
s = 30

filter1 = np.ones(N+1)
#filter2 = np.ones(N)
#filter1 = np.hstack((np.ones(Nc),np.exp(- coef_f*((np.arange(N - Nc + 1))/(N-Nc))**s)))
#filter1 = np.exp(- coef_f*((np.arange(N - Nc + 1))/(N-Nc))**s)

In [ ]:
# Runge Kutta 4th order

h = 0.0002   # step size
tf = 400

It = int(tf/h)

#t1 = np.linspace(0, tf, It)

t = 0

Madm = []
Alpha_origin = []
phi_origin = []
L2HC = []
phi_set = []

out_a = open('Alpha_origin.txt', 'a')
out_a.truncate(0)                        # arase the old datas and start the new ones

out_p = open('phi_origin.txt', 'a')
out_p.truncate(0)


M0 = 2*np.dot(np.arange(1, 2*N + 2, 2), c0) # Madm(t = 0)


#matriz = np.concatenate((array1, array2))

while t <= tf:

  # First step
  Phi = np.dot(a0, psi)
  rPhi = np.dot(a0, rpsi)
  rrPhi = np.dot(a0, rrpsi)
  Pi = np.dot(b0, psi)
  rPi= np.dot(b0, rpsi)
  Chi = np.dot(c0, psi)
  rChi = np.dot(c0, rpsi)
  rrChi = np.dot(c0, rrpsi)
  Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
  inv_matrix_krr = np.linalg.inv(Matrix_Krr)
  rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
  ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
  Krr = np.dot(ck0, SB1)              # (p+1,)
  rKrr = np.dot(ck0, rSB1)
  Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*(Pi[0]**2 - V(Phi)[0])*psi[:,0]
  Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1]  - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V(Phi)[1:N+1])*psi[:,1:N+1]
  Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
  inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
  rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V(Phi))
  al0 = np.dot(rhsal, inv_matrix_alpha)
  Alpha = 1 + np.dot(al0, psi)
  rAlpha = np.dot(al0, rpsi)
  rrAlpha = np.dot(al0, rrpsi)
  Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
  inv_matrix_beta = np.linalg.inv(Matrix_Beta)
  rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
  be0 = np.dot(rhsbe , inv_matrix_beta)
  Beta = np.dot(be0, SB2)
  rBeta = np.dot(be0, rSB2)
  db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0] - Alpha[0]*dV(Phi)[0]
  db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]*dV(Phi)[1:N+1]
  db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
  dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)
  da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)
  K1 = h*(dc)
  L1 = h*(da)
  N1 = h*(db)

  # L2-error associated to the Hamiltonian constraint
  qPhi = np.dot(a0, qpsi)
  rqPhi= np.dot(a0, rqpsi)
  qPi = np.dot(b0, qpsi)
  qChi = np.dot(c0, qpsi)
  rqChi = np.dot(c0, rqpsi)
  rrqChi = np.dot(c0, rrqpsi)
  qKrr = np.dot(ck0, qSB1)
  H = 4*rqChi**2 + 4*rrqChi + 8*rqChi/rq + 3/4*np.exp(-4*qChi)*qKrr**2 + np.exp(4*qChi)*(1/2*qPi**2 + np.exp(-4*qChi)/2*rqPhi**2)   # Hamiltonian constraint (HC)
  L2HC.append((1/2*np.dot(H**2,wq_col))**1/2)    # L2 error of HC

  # Alpha origin
  Alpha_0 = 1 + np.dot(al0, psi[:,0])
  Alpha_origin.append(Alpha_0)                   # = Alphacenter in matlab
  out_a.write(str(t) + " " + str(Alpha_0))
  out_a.write(',\n')

  # Phi origin:
  phi_0 = np.dot(a0, psi[:,0])
  phi_origin.append(phi_0)
  out_p.write(str(t) + " " + str(phi_0))
  out_p.write(',\n')

  # ADM mass:
  Madm.append(2*np.dot(np.arange(1, 2*N + 2, 2), c0))

  # Second step
  Phi = np.dot(a0 + L1/2, psi)
  rPhi= np.dot(a0 + L1/2 , rpsi)
  rrPhi = np.dot(a0 + L1/2, rrpsi)
  Pi = np.dot(b0 + N1/2, psi)
  rPi= np.dot(b0 + N1/2, rpsi)
  Chi = np.dot(c0 + K1/2, psi)
  rChi = np.dot(c0 + K1/2, rpsi)
  rrChi = np.dot(c0 + K1/2, rrpsi)
  Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
  inv_matrix_krr = np.linalg.inv(Matrix_Krr)
  rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
  ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
  Krr = np.dot(ck0, SB1)              # (p+1,)
  rKrr = np.dot(ck0, rSB1)
  Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*(Pi[0]**2 - V(Phi)[0])*psi[:,0]
  Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V(Phi)[1:N+1])*psi[:,1:N+1]
  Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
  inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
  rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V(Phi))
  al0 = np.dot(rhsal, inv_matrix_alpha)
  Alpha = 1 + np.dot(al0, psi)
  rAlpha = np.dot(al0, rpsi)
  rrAlpha = np.dot(al0, rrpsi)
  Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
  inv_matrix_beta = np.linalg.inv(Matrix_Beta)
  rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
  be0 = np.dot(rhsbe , inv_matrix_beta)
  Beta = np.dot(be0, SB2)
  rBeta = np.dot(be0, rSB2)
  db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0] - Alpha[0]*dV(Phi)[0]
  db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]*dV(Phi)[1:N+1]
  db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
  dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)
  da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)
  K2 = h*(dc)
  L2 = h*(da)
  N2 = h*(db)

  # Third step
  Phi = np.dot(a0 + L2/2, psi)
  rPhi = np.dot(a0 + L2/2 , rpsi)
  rrPhi = np.dot(a0 + L2/2, rrpsi)
  Pi = np.dot(b0 + N2/2, psi)
  rPi= np.dot(b0 + N2/2, rpsi)
  Chi = np.dot(c0 + K2/2, psi)
  rChi = np.dot(c0 + K2/2, rpsi)
  rrChi = np.dot(c0 + K2/2, rrpsi)
  Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
  inv_matrix_krr = np.linalg.inv(Matrix_Krr)
  rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
  ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
  Krr = np.dot(ck0, SB1)              # (p+1,)
  rKrr = np.dot(ck0, rSB1)
  Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*(Pi[0]**2 - V(Phi)[0])*psi[:,0]
  Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V(Phi)[1:N+1])*psi[:,1:N+1]
  Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
  inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
  rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V(Phi))
  al0 = np.dot(rhsal, inv_matrix_alpha)
  Alpha = 1 + np.dot(al0, psi)
  rAlpha = np.dot(al0, rpsi)
  rrAlpha = np.dot(al0, rrpsi)
  Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
  inv_matrix_beta = np.linalg.inv(Matrix_Beta)
  rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
  be0 = np.dot(rhsbe , inv_matrix_beta)
  Beta = np.dot(be0, SB2)
  rBeta = np.dot(be0, rSB2)
  db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0] - Alpha[0]*dV(Phi)[0]
  db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]*dV(Phi)[1:N+1]
  db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
  dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)
  da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)
  K3 = h*(dc)
  L3 = h*(da)
  N3 = h*(db)

  # Forth step
  Phi = np.dot(a0 + L3, psi)
  rPhi= np.dot(a0 + L3 , rpsi)
  rrPhi = np.dot(a0 + L3, rrpsi)
  Pi = np.dot(b0 + N3, psi)
  rPi= np.dot(b0 + N3, rpsi)
  Chi = np.dot(c0 + K3, psi)
  rChi = np.dot(c0 + K3, rpsi)
  rrChi = np.dot(c0 + K3, rrpsi)
  Matrix_Krr = 2*rChi[1:N+1]*SB1[:,1:N+1] + rSB1[:,1:N+1] + 3/r[1:N+1]*SB1[:,1:N+1]   # (p, p)
  inv_matrix_krr = np.linalg.inv(Matrix_Krr)
  rhsk = - Pi[1:N+1]*rPhi[1:N+1]*np.exp(4*Chi[1:N+1]) # (p,)
  ck0 = np.dot(rhsk, inv_matrix_krr)  # (p,)
  Krr = np.dot(ck0, SB1)              # (p+1,)
  rKrr = np.dot(ck0, rSB1)
  Matrix_Alpha_0 = rrpsi[:,0] + 2*coeforigin - np.exp(4*Chi[0])*(Pi[0]**2 - V(Phi)[0])*psi[:,0]
  Matrix_Alpha_int = rrpsi[:,1:N+1] + 2*(1/r[1:N+1] + rChi[1:N+1])*rpsi[:,1:N+1] - 3/2*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]**2*psi[:,1:N+1] - np.exp(4*Chi[1:N+1])*(Pi[1:N+1]**2 - V(Phi)[1:N+1])*psi[:,1:N+1]
  Matrix_Alpha = np.column_stack((Matrix_Alpha_0, Matrix_Alpha_int))
  inv_matrix_alpha = np.linalg.inv(Matrix_Alpha)
  rhsal = 3/2*np.exp(-4*Chi)*Krr**2 + np.exp(4*Chi)*(Pi**2-V(Phi))
  al0 = np.dot(rhsal, inv_matrix_alpha)
  Alpha = 1 + np.dot(al0, psi)
  rAlpha = np.dot(al0, rpsi)
  rrAlpha = np.dot(al0, rrpsi)
  Matrix_Beta = rSB2[:,1:N+1] - SB2[:,1:N+1]/r[1:N+1]
  inv_matrix_beta = np.linalg.inv(Matrix_Beta)
  rhsbe = 3/2*Alpha[1:N+1]*np.exp(-4*Chi[1:N+1])*Krr[1:N+1]
  be0 = np.dot(rhsbe , inv_matrix_beta)
  Beta = np.dot(be0, SB2)
  rBeta = np.dot(be0, rSB2)
  db_origin = 2*np.exp(-4*Chi[0])*Alpha[0]*np.dot(a0, coeforigin) + np.exp(-4*Chi[0])*Alpha[0]*rrPhi[0] - Alpha[0]*dV(Phi)[0]
  db_int = Beta[1:N+1]*rPi[1:N+1] + np.exp(-4*Chi[1:N+1])*(2*Alpha[1:N+1]/r[1:N+1] + rAlpha[1:N+1] + 2*rChi[1:N+1]*Alpha[1:N+1])*rPhi[1:N+1] + np.exp(-4*Chi[1:N+1])*Alpha[1:N+1]*rrPhi[1:N+1] - Alpha[1:N+1]*dV(Phi)[1:N+1]
  db = np.dot(np.hstack((db_origin, db_int)), inv_psi)
  dc = np.dot(Beta*rChi + rBeta/2 - Alpha/2*np.exp(-4*Chi)*Krr, inv_psi)
  da = np.dot(Alpha*Pi + Beta*rPhi, inv_psi)
  K4 = h*(dc)
  L4 = h*(da)
  N4 = h*(db)

  t = t + h
#  print(t)
#  if t > 9:
#    h=0.0001


  # Evolution functions
  a0 = a0 + 1/6 * (L1 + 2*L2 + 2*L3 + L4)
  b0 = filter1*(b0 + 1/6 * (N1 + 2*N2 + 2*N3 + N4))
  c0 = c0 + 1/6 * (K1 + 2*K2 + 2*K3 + K4)
  phi_set.append(np.dot(a0, psiplot))

  print('t =',t,'rPi =', max(rPi), sep = '\n')


#  pi_set[i,:] = np.dot(b0, psiplot)
#  chi_set[i,:] = np.dot(c0, psiplot)

out_a.close()
out_p.close()

t1 = np.linspace(0, tf, len(Alpha_origin))

t =
0.0002
rPi =
2.993171390978322e-05
t =
0.0004
rPi =
5.986342436369979e-05
t =
0.0006000000000000001
rPi =
8.979512675498245e-05
t =
0.0008
rPi =
0.0001197268165530187
t =
0.001
rPi =
0.00014965848922670724
t =
0.0012000000000000001
rPi =
0.0001795901402443734
t =
0.0014000000000000002
rPi =
0.00020952176507367493
t =
0.0016000000000000003
rPi =
0.00023945335918131825
t =
0.0018000000000000004
rPi =
0.0002693849180330777
t =
0.0020000000000000005
rPi =
0.00029931643709357487
t =
0.0022000000000000006
rPi =
0.0003292479118261881
t =
0.0024000000000000007
rPi =
0.0003591793376928606
t =
0.0026000000000000007
rPi =
0.000389110710154002
t =
0.002800000000000001
rPi =
0.0004190420246683791
t =
0.003000000000000001
rPi =
0.0004489732766929211
t =
0.003200000000000001
rPi =
0.0004789044616826008
t =
0.003400000000000001
rPi =
0.0005088355750903355
t =
0.003600000000000001
rPi =
0.0005387666123668462
t =
0.0038000000000000013
rPi =
0.0005686975689604387
t =
0.004000000000000001
rPi =
0.0005

t =
0.032399999999999915
rPi =
0.004845619451981279
t =
0.032599999999999914
rPi =
0.004875487812998433
t =
0.03279999999999991
rPi =
0.00490535534827554
t =
0.03299999999999991
rPi =
0.004935222051776816
t =
0.03319999999999991
rPi =
0.00496508791745174
t =
0.03339999999999991
rPi =
0.0049949529392351475
t =
0.03359999999999991
rPi =
0.0050248171110471385
t =
0.033799999999999907
rPi =
0.0050546804267930885
t =
0.033999999999999905
rPi =
0.005084542880363683
t =
0.034199999999999904
rPi =
0.005114404465634914
t =
0.0343999999999999
rPi =
0.005144265176468033
t =
0.0345999999999999
rPi =
0.005174125006709605
t =
0.0347999999999999
rPi =
0.005203983950191523
t =
0.0349999999999999
rPi =
0.005233842000730891
t =
0.0351999999999999
rPi =
0.00526369915213027
t =
0.0353999999999999
rPi =
0.0052935553981774495
t =
0.035599999999999896
rPi =
0.005323410732645542
t =
0.035799999999999894
rPi =
0.005353265149293063
t =
0.03599999999999989
rPi =
0.0053831186418638296
t =
0.03619999999999989
rPi 

t =
0.06499999999999981
rPi =
0.009698593068450768
t =
0.06519999999999981
rPi =
0.009728237746179908
t =
0.06539999999999982
rPi =
0.00975788045260547
t =
0.06559999999999983
rPi =
0.00978752118010154
t =
0.06579999999999983
rPi =
0.009817159921041744
t =
0.06599999999999984
rPi =
0.00984679666779983
t =
0.06619999999999984
rPi =
0.00987643141274954
t =
0.06639999999999985
rPi =
0.00990606414826486
t =
0.06659999999999985
rPi =
0.009935694866720217
t =
0.06679999999999986
rPi =
0.00996532356049059
t =
0.06699999999999987
rPi =
0.009994950221951621
t =
0.06719999999999987
rPi =
0.010024574843479869
t =
0.06739999999999988
rPi =
0.01005419741745292
t =
0.06759999999999988
rPi =
0.010083817936249469
t =
0.06779999999999989
rPi =
0.010113436392249632
t =
0.0679999999999999
rPi =
0.010143052777835025
t =
0.0681999999999999
rPi =
0.010172667085388765
t =
0.0683999999999999
rPi =
0.01020227930729602
t =
0.06859999999999991
rPi =
0.010231889435943714
t =
0.06879999999999992
rPi =
0.0102614974

t =
0.0994000000000008
rPi =
0.014762307316219308
t =
0.0996000000000008
rPi =
0.014791505969313595
t =
0.0998000000000008
rPi =
0.014820701460514597
t =
0.10000000000000081
rPi =
0.014849893784242703
t =
0.10020000000000082
rPi =
0.014879082934938923
t =
0.10040000000000082
rPi =
0.0149082689070651
t =
0.10060000000000083
rPi =
0.014937451695103764
t =
0.10080000000000083
rPi =
0.014966631293558328
t =
0.10100000000000084
rPi =
0.014995807696952936
t =
0.10120000000000084
rPi =
0.015024980899832674
t =
0.10140000000000085
rPi =
0.01505415089676347
t =
0.10160000000000086
rPi =
0.01508331768233228
t =
0.10180000000000086
rPi =
0.015112481251146878
t =
0.10200000000000087
rPi =
0.01514164159783607
t =
0.10220000000000087
rPi =
0.01517079871704974
t =
0.10240000000000088
rPi =
0.015199952603458695
t =
0.10260000000000088
rPi =
0.015229103251754885
t =
0.10280000000000089
rPi =
0.015258250656651222
t =
0.1030000000000009
rPi =
0.015287394812881843
t =
0.1032000000000009
rPi =
0.0153165357

t =
0.1344000000000018
rPi =
0.01981980975848555
t =
0.1346000000000018
rPi =
0.019848387850646052
t =
0.1348000000000018
rPi =
0.019876962098255572
t =
0.1350000000000018
rPi =
0.01990553249868328
t =
0.13520000000000182
rPi =
0.019934099049305033
t =
0.13540000000000182
rPi =
0.019962661747502778
t =
0.13560000000000183
rPi =
0.01999122059066491
t =
0.13580000000000184
rPi =
0.020019775576185346
t =
0.13600000000000184
rPi =
0.02004832670146403
t =
0.13620000000000185
rPi =
0.020076873963906415
t =
0.13640000000000185
rPi =
0.02010541736092347
t =
0.13660000000000186
rPi =
0.020133956889931227
t =
0.13680000000000186
rPi =
0.020162492548350876
t =
0.13700000000000187
rPi =
0.02019102433360851
t =
0.13720000000000188
rPi =
0.020219552243134757
t =
0.13740000000000188
rPi =
0.02024807627436502
t =
0.1376000000000019
rPi =
0.02027659642473878
t =
0.1378000000000019
rPi =
0.020305112691699934
t =
0.1380000000000019
rPi =
0.02033362507269611
t =
0.1382000000000019
rPi =
0.0203621335651788

...

t =
5.2596000000022505
rPi =
0.20908506719354464
t =
5.259800000002251
rPi =
0.2090981494395759
t =
5.260000000002251
rPi =
0.2091112307569033
t =
5.260200000002252
rPi =
0.2091243111447751
t =
5.260400000002252
rPi =
0.20913739060243955
t =
5.260600000002253
rPi =
0.20915046912914972
t =
5.260800000002253
rPi =
0.20916354672415866
t =
5.261000000002253
rPi =
0.20917662338672213
t =
5.261200000002254
rPi =
0.2091896991160983
t =
5.261400000002254
rPi =
0.20920277391154618
t =
5.261600000002255
rPi =
0.20921584777232838
t =
5.261800000002255
rPi =
0.20922892069770826
t =
5.2620000000022555
rPi =
0.2092419926869517
t =
5.262200000002256
rPi =
0.2092550637393261
t =
5.262400000002256
rPi =
0.20926813385410126
t =
5.262600000002257
rPi =
0.20928120303054867
t =
5.262800000002257
rPi =
0.20929427126794184
t =
5.263000000002258
rPi =
0.20930733856555708
t =
5.263200000002258
rPi =
0.20932040492266984
t =
5.2634000000022585
rPi =
0.20933347033856103
t =
5.263600000002259
rPi =
0.2093465348125

t =
5.295000000002325
rPi =
0.2113915173498336
t =
5.295200000002326
rPi =
0.2114063811077174
t =
5.295400000002326
rPi =
0.2114212457002972
t =
5.2956000000023264
rPi =
0.2114361111274075
t =
5.295800000002327
rPi =
0.21145097738888255
t =
5.296000000002327
rPi =
0.21146584448455397
t =
5.296200000002328
rPi =
0.2114807124142533
t =
5.296400000002328
rPi =
0.21149558117780967
t =
5.296600000002329
rPi =
0.21151045077505176
t =
5.296800000002329
rPi =
0.21152532120580497
t =
5.297000000002329
rPi =
0.21154019246989642
t =
5.29720000000233
rPi =
0.2115550645671489
t =
5.29740000000233
rPi =
0.21156993749738548
t =
5.297600000002331
rPi =
0.2115848112604272
t =
5.297800000002331
rPi =
0.21159968585609384
t =
5.2980000000023315
rPi =
0.21161456128420422
t =
5.298200000002332
rPi =
0.21162943754457514
t =
5.298400000002332
rPi =
0.2116443146370221
t =
5.298600000002333
rPi =
0.21165919256136015
t =
5.298800000002333
rPi =
0.21167407131740193
t =
5.299000000002334
rPi =
0.21168895090495873


t =
5.329400000002398
rPi =
0.21396017264143455
t =
5.329600000002398
rPi =
0.21397517638128924
t =
5.329800000002399
rPi =
0.21399018090661415
t =
5.330000000002399
rPi =
0.21400518621698625
t =
5.3302000000023995
rPi =
0.21402019231198122
t =
5.3304000000024
rPi =
0.21403519919117264
t =
5.3306000000024
rPi =
0.21405020685413317
t =
5.330800000002401
rPi =
0.21406521530043224
t =
5.331000000002401
rPi =
0.21408022452963807
t =
5.331200000002402
rPi =
0.2140952345413168
t =
5.331400000002402
rPi =
0.21411024533503326
t =
5.331600000002402
rPi =
0.2141252569103492
t =
5.331800000002403
rPi =
0.21414026926682542
t =
5.332000000002403
rPi =
0.21415528240402013
t =
5.332200000002404
rPi =
0.2141702963214903
t =
5.332400000002404
rPi =
0.2141853110187902
t =
5.3326000000024045
rPi =
0.21420032649547266
t =
5.332800000002405
rPi =
0.21421534275108864
t =
5.333000000002405
rPi =
0.2142303597851866
t =
5.333200000002406
rPi =
0.21424537759731307
t =
5.333400000002406
rPi =
0.21426039618701334

t =
5.366200000002475
rPi =
0.21673355251802187
t =
5.366400000002476
rPi =
0.21674869134612046
t =
5.366600000002476
rPi =
0.2167638308408863
t =
5.366800000002477
rPi =
0.21677897100138466
t =
5.367000000002477
rPi =
0.21679411182667646
t =
5.3672000000024775
rPi =
0.2168092533158192
t =
5.367400000002478
rPi =
0.21682439546786647
t =
5.367600000002478
rPi =
0.21683953828186803
t =
5.367800000002479
rPi =
0.21685468175686917
t =
5.368000000002479
rPi =
0.21686982589191134
t =
5.36820000000248
rPi =
0.21688497068603238
t =
5.36840000000248
rPi =
0.21690011613826543
t =
5.3686000000024805
rPi =
0.21691526224764104
t =
5.368800000002481
rPi =
0.21693040901318422
t =
5.369000000002481
rPi =
0.21694555643391647
t =
5.369200000002482
rPi =
0.2169607045088548
t =
5.369400000002482
rPi =
0.21697585323701343
t =
5.369600000002483
rPi =
0.21699100261740156
t =
5.369800000002483
rPi =
0.21700615264902368
t =
5.3700000000024835
rPi =
0.21702130333088157
t =
5.370200000002484
rPi =
0.217036454661

t =
5.402800000002553
rPi =
0.21951392231407896
t =
5.403000000002553
rPi =
0.21952916303990064
t =
5.4032000000025535
rPi =
0.21954440418280216
t =
5.403400000002554
rPi =
0.21955964574091558
t =
5.403600000002554
rPi =
0.21957488771236552
t =
5.403800000002555
rPi =
0.21959013009527031
t =
5.404000000002555
rPi =
0.21960537288774223
t =
5.404200000002556
rPi =
0.21962061608788855
t =
5.404400000002556
rPi =
0.2196358596938084
t =
5.4046000000025565
rPi =
0.21965110370359614
t =
5.404800000002557
rPi =
0.21966634811533906
t =
5.405000000002557
rPi =
0.21968159292711767
t =
5.405200000002558
rPi =
0.2196968381370083
t =
5.405400000002558
rPi =
0.21971208374307927
t =
5.405600000002559
rPi =
0.2197273297433924
t =
5.405800000002559
rPi =
0.21974257613600465
t =
5.406000000002559
rPi =
0.21975782291896573
t =
5.40620000000256
rPi =
0.2197730700903189
t =
5.40640000000256
rPi =
0.2197883176481018
t =
5.406600000002561
rPi =
0.21980356559034497
t =
5.406800000002561
rPi =
0.219818813915073

In [ ]:
# Searching for critical amplitude:

plt.figure(figsize = (10,5))

plt.plot(t1, Alpha_origin, color = "g", label = "$R_0$ = {:}".format(R0))
#plt.title("Alpha na origem para L0 = 15, N = {:}".format(N))
plt.ylabel(r"$\alpha(t,0)$")
plt.xlabel("t")
plt.legend()

In [ ]:
plt.figure(figsize = (10,5))

plt.plot(t1, phi_origin, color = "b", label = "$R_0$ = {:}".format(R0))
#plt.title("Phi na origem para L0 = 2 e N = {:}".format(N) )
plt.ylabel("$\phi$(t,0)")
plt.xlabel("t")
#plt.xlim(7.2,8.2)
plt.legend()

In [ ]:
# Mass ADM error

Madm_error = abs(Madm - M0)/M0

plt.plot(t1, Madm_error, color = "r", label ="R_0 = {:}".format(R0))
plt.title("Mass ADM para L0 = 1 e N = {:}".format(N) )
plt.ylabel("Madm")
plt.xlabel("t")
#plt.xlim(7.2,8.2)

plt.legend()

In [ ]:


# Animation plot for Phi: Scalar Field

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

#fig = plt.figure()
ax = plt.axes(xlim=(0, 15),ylim = (-3, 3))
line, = ax.plot([], [], lw=2)
initA0_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
x = rplot
num_points_to_show = len(rplot)//4  # Mostrar 1/4 dos pontos para acelerar a animação

def init():
    line.set_data([], [])
    initA0_text.set_text('')
    return line,

def animate(i):
  y = phi_set[i]
  x_subset = x[::num_points_to_show] # Selecionar um subconjunto de pontos
  y_subset = y[::num_points_to_show]
  line.set_data(x_subset, y_subset)
  initA0_text.set_text("$R_0 = {:}$".format(R0))
  return line,

anim = FuncAnimation(fig, animate, init_func=init,
                               frames= It, interval=0.2, blit=True)

#HTML(anim.to_html5_video())
anim.save('animation_N200_Potential.mp4')